In [5]:
from PIL import Image, ImageDraw, ImageEnhance
import random
import os
from tqdm import tqdm


def modify_color_channel(image, mode):
    if mode == 'RGB':
        r, g, b = image.split()

        # Modify color channels as before
        modified_r = r.point(lambda i: i * random.uniform(0.5, 1.5))
        modified_g = g.point(lambda i: i * random.uniform(0.5, 1.5))
        modified_b = b.point(lambda i: i * random.uniform(0.5, 1.5))

        return Image.merge("RGB", (modified_r, modified_g, modified_b))
    elif mode == 'RGBA':
        r, g, b, a = image.split()
        # Modify color channels as before
        modified_r = r.point(lambda i: i * random.uniform(0.5, 1.5))
        modified_g = g.point(lambda i: i * random.uniform(0.5, 1.5))
        modified_b = b.point(lambda i: i * random.uniform(0.5, 1.5))

        return Image.merge("RGBA", (modified_r, modified_g, modified_b, a))

def process_images(label_int, background_path, img_path, output_path, txt_output_path, enter):
    # Open the background and image images
    background = Image.open(background_path)
    img = Image.open(img_path)

    # Check the image mode and modify color channels accordingly
    if img.mode == 'RGB':
        img = modify_color_channel(img, 'RGB')
    elif img.mode == 'RGBA':
        img = modify_color_channel(img, 'RGBA')

    # Resize the image to a random size between 1/20 and 1/10 of the background image
    target_size = random.randint(background.width // 30, background.width // 5)
    img = img.resize((target_size, target_size), Image.ANTIALIAS)

    # Randomly rotate the image
    rotation_angle = random.randint(0, 360)
    img = img.rotate(rotation_angle, expand=True)

    # Randomly change the color using ImageEnhance
    color_factor = random.uniform(0.5, 1.5)
    img = ImageEnhance.Color(img).enhance(color_factor)

    # Paste the resized and rotated image onto the background
    position = (random.randint(0, background.width - img.width), random.randint(0, background.height - img.height))
    background.paste(img, position, img)

    paste_position = position
    # Calculate the min and max positions of the pasted image
    min_position = paste_position
    max_position = (paste_position[0] + img.width, paste_position[1] + img.height)

    #print(min_position, max_position)
    # Draw bounding box around pasted image
    #draw = ImageDraw.Draw(background)
    #bbox = [paste_position[0], paste_position[1], max_position[0], max_position[1]]
    #draw.rectangle(bbox, outline="red")

    # Calculate the center coordinates of the bounding box
    center_x = (min_position[0] + max_position[0]) / 2 / background.width
    center_y = (min_position[1] + max_position[1]) / 2 / background.height

    # Calculate the normalized width and height of the bounding box
    bbox_width = (max_position[0] - min_position[0]) / background.width
    bbox_height = (max_position[1] - min_position[1]) / background.height
    
    
    # Save the YOLO format bounding box information to a text file
    if enter == False:
        yolo_bbox_info = f"{label_int} {center_x:.6f} {center_y:.6f} {bbox_width:.6f} {bbox_height:.6f}\n"
    else:
        yolo_bbox_info = f"\n{label_int} {center_x:.6f} {center_y:.6f} {bbox_width:.6f} {bbox_height:.6f}"
        
    with open(txt_output_path, "a") as txt_file:
        txt_file.write(yolo_bbox_info)

    # Save the final image
    background.save(output_path)
    #background.show()




review the variable carefully before run

In [6]:
#image_folder  = "../dataset/training_data/images/train/"
image_folder  = r"C:\Users\Administrator\Downloads\23_08"
#label_folder = "../dataset/training_data/labels/train/"
label_folder = r"C:\Users\Administrator\Downloads\23_08_labels"
chinese_png_folder = "../dataset/chinese_map_png/"
chinese_png_folder = "../dataset/nine_dash/"
label_int = 3  # Modify this label as needed
chinese_filename_list = os.listdir(chinese_png_folder)
count = 0
except_count = 0
for bg_filename in tqdm(os.listdir(image_folder)):
    if bg_filename.startswith('.'): continue
    img_type = bg_filename.split('.')[-1]
    bg_path = os.path.join(image_folder, bg_filename)

    try:
        #print(f"Unsupported image mode: {image.mode}")
    
        main_img_path = bg_path
        main_img = Image.open(main_img_path)
        main_image_size = main_img.size
        main_img.close()
        if main_image_size[0] <300 or main_image_size[1] <300:
            continue
        png_file = chinese_filename_list[random.randint(0, len(chinese_filename_list)-1)]
        png_img_path = os.path.join(chinese_png_folder, png_file)
     
    
        #os.remove(main_img_path)
        pasted_output_path = main_img_path
        txt_output_path = os.path.join(label_folder, bg_filename.replace(img_type, 'txt'))
        
        if os.path.exists(txt_output_path):
            enter = False
        else:
            enter = False
    
        # Process images
        process_images(label_int, main_img_path, png_img_path, pasted_output_path, txt_output_path, enter)
        count+=1
    
    except Exception as e:
        except_count+=1
        pass
        #print("Error:", image.mode)
print(count)
print(except_count)

  0%|                                                                                          | 0/450 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_3972\2185282396.py:39: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((target_size, target_size), Image.ANTIALIAS)
100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:23<00:00, 19.52it/s]

385
0
